In [394]:
import pandas as pd
from pathlib import Path

In [395]:
school_data_path = Path("Resources", "schools_complete.csv")
student_data_path = Path("Resources", "students_complete.csv")

In [396]:
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

district_df = pd.merge(school_data,student_data, how="left", on="school_name")
district_df.head()


,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [397]:
# Total number of unique schools

unique_schools = district_df["school_name"].unique()
unique_schools_count = len(unique_schools)
unique_schools_count


15

In [398]:
#Total Students

student_count = len(district_df['student_name'])

In [399]:
#Total budget

no_duplicates_district_df = district_df['budget'].drop_duplicates()
budget = no_duplicates_district_df.sum()
budget


24649428

In [400]:
#Calculate the average math score

average_math = district_df['math_score'].mean()
average_math

78.98537145774827

In [401]:
#Calculate the average reading score

average_read = district_df['reading_score'].mean()
average_read

81.87784018381414

In [402]:
#Calculate the average math score percent

passing_math = district_df[(district_df['math_score']>= 70)].count()['student_name']
passing_math_pct = passing_math / float(student_count) * 100
passing_math_pct

74.9808526933878

In [403]:
#Calculate the average reading score percent

passing_read = district_df[(district_df['reading_score']>= 70)].count()['student_name']
passing_read_pct = passing_read / float(student_count) * 100
passing_read_pct

85.80546336482001

In [404]:
# % overall passing (the percentage of students who passed math AND reading)

passing_math_read = district_df[(district_df['math_score']>= 70) & (district_df['reading_score']>= 70)].count()['student_name']
passing_math_read_pct = passing_math_read / float(student_count) * 100
passing_math_read_pct

65.17232575950983

## District Summary

In [405]:
#Create a DataFrame that summarizes key metrics about each school.

district_summary = {'Total Schools':[len(unique_schools)],'Total Students':[student_count],
                     'Total Budget':[budget],'Average Math Score':[average_math],'Average Reading Score':[average_read],
                     '% Passing Math':[passing_math_pct],'% Passing Reading':[passing_read_pct],
                     '% Overall Passing':[passing_math_read_pct]}
district_summary_df = pd.DataFrame(district_summary)
district_summary_df['Total Students'] = district_summary_df['Total Students'].map("{:,}".format)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map("${:,}".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [406]:
school_type_df = district_df[['school_name','type']]
school_type_df

,school_name,type
0,Huang High School,District
1,Huang High School,District
2,Huang High School,District
3,Huang High School,District
4,Huang High School,District
...,...,...
39165,Thomas High School,Charter
39166,Thomas High School,Charter
39167,Thomas High School,Charter
39168,Thomas High School,Charter


In [407]:
#calculations of per school numbers

stu_per_school = district_df.groupby('school_name')['student_name'].count()
stu_per_school_df = pd.DataFrame(stu_per_school)
budget_per_school = school_data.groupby('school_name')['budget'].sum()
budget_per_school_df = pd.DataFrame(budget_per_school)

avg_math_per_school = district_df.groupby('school_name')['math_score'].mean()
avg_math_per_school_df = pd.DataFrame(avg_math_per_school)

avg_read_per_school = district_df.groupby('school_name')['reading_score'].mean()
avg_read_per_school_df = pd.DataFrame(avg_read_per_school)

pct_passing_math_per_school = district_df.groupby('school_name').apply(lambda x:(x['math_score']>= 70).mean()*100)
pct_passing_math_per_school_df = pd.DataFrame(pct_passing_math_per_school)

pct_passing_read_per_school = district_df.groupby('school_name').apply(lambda x:(x['reading_score']>= 70).mean()*100)
pct_passing_read_per_school_df = pd.DataFrame(pct_passing_read_per_school)

avg_by_school = district_df.groupby('school_name').apply(lambda x:((x['math_score']>= 70) & (x['reading_score']>= 70)).mean()*100).reset_index(name='% Overall Passing')
avg_by_school_df = pd.DataFrame(avg_by_school)




In [408]:

per_school_summary = pd.merge(school_type_df,stu_per_school, on='school_name', how='left')
per_school_summary = pd.merge(per_school_summary,budget_per_school_df, on='school_name', how='left')

per_school_summary.rename(columns={'type': 'School Type','student_name': 'Total Students','budget': 'Total School Budget'},inplace=True)

per_school_summary['Per Student Budget'] = per_school_summary['Total School Budget'] / per_school_summary['Total Students']

per_school_summary = pd.merge(per_school_summary,avg_math_per_school_df, on= 'school_name', how='left')
per_school_summary = pd.merge(per_school_summary,avg_read_per_school_df, on= 'school_name', how='left')
per_school_summary.rename(columns={'math_score': 'Average Math Score','reading_score': 'Average Reading Score'},inplace=True)

per_school_summary= pd.merge(per_school_summary,pct_passing_math_per_school_df, on='school_name', how='left')
per_school_summary= pd.merge(per_school_summary,pct_passing_read_per_school_df, on='school_name', how='left')
per_school_summary.rename(columns={'0_x': '% Passing Math','0_y': '% Passing Reading'},inplace=True)

per_school_summary = pd.merge(per_school_summary, avg_by_school_df,on= 'school_name', how='left')
per_school_summary.rename(columns={'school_name': 'School'},inplace=True)


per_school_summary


,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
1,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
2,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
3,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
4,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
...,...,...,...,...,...,...,...,...,...,...
39165,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
39166,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
39167,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
39168,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012


## Highest-Performing Schools (by % Overall Passing)

In [409]:
# Sort the schools by % Overall Passing in descending order and display the top 5 rows.
# Save the results in a DataFrame called "top_schools"

top_schools_df = per_school_summary

top_schools_df = top_schools_df.sort_values('% Overall Passing', ascending = False)
top_schools_df['Total School Budget']= top_schools_df['Total School Budget'].map('${:,.2f}'.format)
top_schools_df['Per Student Budget']= top_schools_df['Per Student Budget'].map('${:,.2f}'.format)

top_schools_df.head(5)


,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
17168,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.97578,94.133477,97.039828,91.334769
17247,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.97578,94.133477,97.039828,91.334769
17259,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.97578,94.133477,97.039828,91.334769
17258,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.97578,94.133477,97.039828,91.334769
17257,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.97578,94.133477,97.039828,91.334769


## Bottom Performing Schools (By % Overall Passing)

In [410]:
# Sort the schools by % Overall Passing in ascending order and display the top 5 rows.
# Save the results in a DataFrame called "bottom_schools".
bottom_schools_df = top_schools_df
bottom_schools_df=bottom_schools_df.sort_values('% Overall Passing', ascending = True)
bottom_schools_df.head(5)

,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
28170,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
30029,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
30028,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
30027,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
30026,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247


## Math Scores by Grade

In [411]:
# Perform the necessary calculations to create a DataFrame that lists the average math score 
# for students of each grade level (9th, 10th, 11th, 12th) at each school.

ninth_graders = district_df[(district_df["grade"] == "9th")]
tenth_graders = district_df[(district_df["grade"] == "10th")]
eleventh_graders = district_df[(district_df["grade"] == "11th")]
twelfth_graders = district_df[(district_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby('school_name')['math_score'].mean()
tenth_grader_math_scores =  tenth_graders.groupby('school_name')['math_score'].mean()
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')['math_score'].mean()
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')['math_score'].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.merge(ninth_grade_math_scores,tenth_grader_math_scores, how='left', on='school_name')
math_scores_by_grade.rename(columns={'math_score_x': '9th Graders','math_score_y':'10th Graders'},inplace=True)

math_scores_by_grade = pd.merge(math_scores_by_grade, eleventh_grader_math_scores, how='left', on='school_name')
math_scores_by_grade.rename(columns={'math_score': '11th Graders'},inplace=True)

math_scores_by_grade = pd.merge(math_scores_by_grade, twelfth_grader_math_scores, how='left', on='school_name')
math_scores_by_grade.rename(columns={'math_score': '12th Graders'},inplace=True)
math_scores_by_grade.rename(columns={'school_name': 'School'},inplace=True)
math_scores_by_grade.index.name = None
math_scores_by_grade

,9th Graders,10th Graders,11th Graders,12th Graders
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [412]:
# Perform the necessary calculations to create a DataFrame that lists the average reading score 
# for students of each grade level (9th, 10th, 11th, 12th) at each school.
ninth_graders = district_df[(district_df["grade"] == "9th")]
tenth_graders = district_df[(district_df["grade"] == "10th")]
eleventh_graders = district_df[(district_df["grade"] == "11th")]
twelfth_graders = district_df[(district_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `reading_score` column for each.
ninth_grade_read_scores = ninth_graders.groupby('school_name')['reading_score'].mean()
tenth_grader_read_scores =  tenth_graders.groupby('school_name')['reading_score'].mean()
eleventh_grader_read_scores = eleventh_graders.groupby('school_name')['reading_score'].mean()
twelfth_grader_read_scores = twelfth_graders.groupby('school_name')['reading_score'].mean()

# Combine each of the scores above into single DataFrame called `read_scores_by_grade`
read_scores_by_grade = pd.merge(ninth_grade_read_scores,tenth_grader_read_scores, how='left', on='school_name')
read_scores_by_grade.rename(columns={'reading_score_x': '9th Graders','reading_score_y':'10th Graders'},inplace=True)

read_scores_by_grade = pd.merge(read_scores_by_grade, eleventh_grader_read_scores, how='left', on='school_name')
read_scores_by_grade.rename(columns={'reading_score': '11th Graders'},inplace=True)

read_scores_by_grade = pd.merge(read_scores_by_grade, twelfth_grader_read_scores, how='left', on='school_name')
read_scores_by_grade.rename(columns={'reading_score': '12th Graders'},inplace=True)
read_scores_by_grade.rename(columns={'school_name': 'School'},inplace=True)
read_scores_by_grade.index.name = None
read_scores_by_grade

,9th Graders,10th Graders,11th Graders,12th Graders
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [413]:
# # Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]


In [414]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [415]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_summary['Per Student Budget'], bins=spending_bins, labels=labels, include_lowest=True)
school_spending_df['Total School Budget'] = school_spending_df['Total School Budget'].map('${:,.2f}'.format)
school_spending_df['Per Student Budget'] = school_spending_df['Per Student Budget'].map('${:,.2f}'.format)
school_spending_df

,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
1,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
2,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
3,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
4,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
...,...,...,...,...,...,...,...,...,...,...,...
39165,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012,$630-645
39166,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012,$630-645
39167,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012,$630-645
39168,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012,$630-645


In [416]:
# Use the following code to then calculate mean scores per spending range.

spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

/var/folders/np/wfx8_61x4sd5qgshvt2nf2sh0000gn/T/ipykernel_27399/3049699728.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
/var/folders/np/wfx8_61x4sd5qgshvt2nf2sh0000gn/T/ipykernel_27399/3049699728.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
/var/folders/np/wfx8_61x4sd5qgshvt2nf2sh0000gn/T/ipykernel_27399/3049699728.py:5: FutureWarning: The default of observed=False

In [417]:
# Use the scores above to create a DataFrame called spending_summary.

spending_summary = pd.merge(spending_math_scores,spending_reading_scores, on='Spending Ranges (Per Student)', how = 'left')
spending_summary = pd.merge(spending_summary,spending_passing_math, on='Spending Ranges (Per Student)', how = 'left')
spending_summary = pd.merge(spending_summary,spending_passing_reading, on='Spending Ranges (Per Student)', how = 'left')
spending_summary = pd.merge(spending_summary,overall_passing_spending, on='Spending Ranges (Per Student)', how = 'left')
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,90.640704
$585-630,79.982873,82.312643,79.109851,88.513145,70.939239
$630-645,77.821056,81.301007,70.623565,82.600247,58.841194
$645-680,77.049297,81.005604,66.230813,81.109397,53.528791


## Scores by School Size

In [418]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [419]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary['Total Students'], bins=size_bins, labels=labels, include_lowest=True)
per_school_summary


,School,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
1,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
2,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
3,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
4,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
...,...,...,...,...,...,...,...,...,...,...,...
39165,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012,Medium (1000-2000)
39166,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012,Medium (1000-2000)
39167,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012,Medium (1000-2000)
39168,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012,Medium (1000-2000)


In [420]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

/var/folders/np/wfx8_61x4sd5qgshvt2nf2sh0000gn/T/ipykernel_27399/2584247256.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
/var/folders/np/wfx8_61x4sd5qgshvt2nf2sh0000gn/T/ipykernel_27399/2584247256.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
/var/folders/np/wfx8_61x4sd5qgshvt2nf2sh0000gn/T/ipykernel_27399/2584247256.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True i

In [421]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.merge(size_math_scores,size_reading_scores, on='School Size', how='left')
size_summary = pd.merge(size_summary,size_passing_math, on = 'School Size', how='left')
size_summary = pd.merge(size_summary,size_passing_reading, on = 'School Size', how='left')
size_summary = pd.merge(size_summary,size_overall_passing, on = 'School Size', how='left')

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


## Scores by School Type

In [422]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [423]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.merge(average_math_score_by_type,average_reading_score_by_type, on='School Type', how='left')
type_summary = pd.merge(type_summary,average_percent_passing_math_by_type, on='School Type', how='left')
type_summary = pd.merge(type_summary,average_percent_passing_reading_by_type, on='School Type', how='left')
type_summary = pd.merge(type_summary,average_percent_overall_passing_by_type, on='School Type', how='left')

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878
